# Hugging Face: transferindo aprendizado de modelos de NLP

## 01. Explorando o modelo pré-treinado

### Utilizando o Hugging Face pipeline

In [ ]:
from transformers import pipeline

In [ ]:
classificador = pipeline('zero-shot-classification', model='Mel-Iza0/zero-shot', tokenizer='Mel-Iza0/zero-shot')

In [ ]:
texto = """
Com o início da era digital, a capacidade de transmissão de informações cresceu apressuradamente, 
o que facilitou o contato com diversos assuntos, dentre eles a educação sexual . 
Entretanto, surgiram paralelamente algumas questões, das quais se destacam a preocupação 
com o momento adequado do ingresso do tema a vida do estudante, assim como de maneira antagônica, 
o aumento de casos de DST´S\\xa0\\xa0e gravidez indesejada nesse período, a qual leva a um maior 
questionamento sobre o começo desta pauta., A falta de comunicação sobre a sexualidade 
entre jovens no Brasil acarreta muitas das vezes\\xa0na inserção desses em um meio 
repleto de dúvidas, gerando a ocorrência de doenças sexualmente transmissíveis e de gravidez precoce. 
Com base nisso, muitos adolescentes buscam compreender melhor essas questões na internet, 
local onde se podem encontrar notícias falsas ou inadequadas para seu desenvolvimento, 
impedindo assim a correta compreensão do assunto, assim como a responsabilidade imposta por ele.
, Por outro lado, o diálogo em relação à sexualidade e seus tópicos é um tabu para pais e professores, 
que se sentem desorientados sobre a devida hora e os devidos critérios a serem tratados 
com os filhos e alunos, dificultando com que esses esclareçam suas dúvidas e entenda de maneira correta, 
o que levaria a conscientização da seriedade dessa discussão., Em virtude do que foi mencionado, 
as indagações a respeito divide várias opiniões e reflexões acima do que deve ser feito. 
É de extrema importância o Ministério da Educação, em parceria com o Ministério da Cidadania, 
implantar a educação sexual na matriz curricular estudantil dos jovens, através de aulas elaboradas 
e destinadas ao esclarecimento de perguntas, assim como palestras e programas com a intenção de 
propagar o conteúdo aos estudantes, contando com o suporte dos pais, funcionários e encarregados 
da rede de ensino no país, para que seja realizado constantemente.
"""

In [ ]:
classificador(texto, candidate_labels=[str(i) for i in range(11)])['labels'][0]

### Para saber mais: aprendizado Zero-Shot

Normalmente, modelos de IA que classificam dados em categorias são treinados de forma supervisionada, utilizando uma base de dados que contém os rótulos a serem previstos. Esses modelos supervisionados se tornam especialistas em uma tarefa específica, mas, quando surge um novo rótulo, eles não sabem como classificá-lo. Nesse caso, seria necessário treinar um novo modelo do zero, incorporando o novo rótulo na variável alvo.

#### Alternativa aos modelos convencionais

Uma alternativa promissora para contornar essa limitação é o Zero-Shot Learning (Aprendizado Zero-Shot). Este tipo de aprendizado permite que modelos de IA reconheçam e classifiquem novos rótulos que não estavam presentes durante o treinamento inicial. Isso é possível graças ao uso de descrições ou atributos dos novos rótulos, que são aprendidos de maneira a permitir que o modelo generalize para categorias inéditas.

No aprendizado Zero-Shot, os modelos são treinados com uma combinação de dados rotulados e metadados que descrevem os rótulos. Por exemplo, em uma tarefa de classificação de imagens, o modelo pode ser treinado não apenas com imagens de gatos e cavalos, mas também com descrições textuais dessas classes. Assim, quando o modelo se depara com uma nova classe, como “tigre”, ele pode usar o conhecimento das descrições textuais para inferir as características do tigre, mesmo sem ter visto imagens dessa classe durante o treinamento.

![Aprendizado Zero-Shot](http://cdn3.gnarususercontent.com.br/3977-hugging-face/Imagens%20das%20atividades/ZERO%20SHOT.png)

Essa abordagem é particularmente útil em domínios onde é impraticável ou impossível obter exemplos rotulados para todas as categorias possíveis. Em áreas como diagnóstico médico, reconhecimento de objetos ou processamento de linguagem natural, novas classes de dados podem surgir frequentemente, e o Zero-Shot oferece uma maneira eficiente de lidar com essas situações sem a necessidade de re-treinar modelos extensivamente.

Além disso, o Aprendizado Zero-Shot pode ser complementado com técnicas como o Few-Shot Learning, onde o modelo é adaptado para aprender novas classes com um número muito pequeno de exemplos rotulados. Juntas, essas abordagens ampliam a capacidade dos modelos de IA de se adaptarem a novos desafios, tornando-os mais flexíveis e robustos em ambientes dinâmicos e em constante mudança.

Um grande desafio enfrentado na utilização deste tipo de aprendizado está na dificuldade em lidar com a variabilidade que pode ocorrer dos conceitos que não foram vistos durante o treinamento. Como o modelo não teve contato com exemplos desses conceitos, ele pode ter dificuldades em fazer previsões precisas.

Em resumo, o Zero-Shot Learning constitui um avanço importante no campo da inteligência artificial, permitindo que modelos identifiquem e classifiquem novas categorias sem depender de grandes quantidades de dados rotulados. Essa habilidade de generalização é essencial para criar sistemas de IA mais versáteis e escaláveis, capazes de lidar com a diversidade e a complexidade do mundo real.

### Realizando o deploy com o Gradio

In [ ]:
from typing import Any
import gradio as gr
import json

In [ ]:
def mostrar_resultado(texto: str) -> dict[str, Any]:
    return classificador(texto, candidate_labels=[str(i) for i in range(11)])['labels'][0]

In [ ]:
# app = gr.Interface(
#     fn=mostrar_resultado,
#     inputs=['text'],
#     outputs=['text']
# )
# app.launch(debug=True)

## 02. Preparando dados de texto

### Carregando a base de dados

In [ ]:
from datasets import load_dataset

In [ ]:
dados_redacoes = load_dataset('csv', data_files='./redacoes.csv')

In [ ]:
dados_redacoes

In [ ]:
dados_redacoes['train'].features

In [ ]:
dados_redacoes['train']['essay'][0]

In [ ]:
dados_redacoes['train']['score'][0]

In [ ]:
dados_redacoes['train'].to_pandas()

### Separando os dados

In [ ]:
treino_teste = dados_redacoes['train'].train_test_split(test_size=0.2, shuffle=False)
treino_teste

In [ ]:
from datasets import DatasetDict

In [ ]:
dados_redacoes = DatasetDict({
    'treino': treino_teste['train'],
    'teste': treino_teste['test']
})

In [ ]:
dados_redacoes

[Datasets - Hugging Face](https://huggingface.co/datasets)

### Tokenizando dados textuais

In [ ]:
checkpoint_modelo = 'Geotrend/distilbert-base-pt-cased'

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizador = AutoTokenizer.from_pretrained(checkpoint_modelo)

In [ ]:
texto = dados_redacoes['treino']['essay'][0]

In [ ]:
tokens = tokenizador.tokenize(texto)
print(tokens)

In [ ]:
ids = tokenizador.convert_tokens_to_ids(tokens)
print(ids)

In [ ]:
texto_decodificado = tokenizador.decode(ids)
print(texto_decodificado)

In [ ]:
input_codificado = tokenizador(texto, return_tensors='tf')
print(input_codificado)

### Tokenizando a base de dados

In [ ]:
def funcao_tokenizadora(dados_texto):
    return tokenizador(dados_texto['essay'], truncation=True)

In [ ]:
dataset_tokenizado = dados_redacoes.map(funcao_tokenizadora, batched=True, remove_columns=['essay'])

In [ ]:
dataset_tokenizado

In [ ]:
dataset_tokenizado['treino'].to_pandas()

In [ ]:
dataset_tokenizado['teste'].to_pandas()

### Para saber mais: Padding e Truncation

Na utilização de dados textais, é muito comum que os textos tenham tamanhos variados. No entanto, as redes neurais geralmente exigem que os tensores utilizados como entradas tenham um tamanho fixo. Assim, ao converter textos em tokens, as entradas em lote podem ter comprimentos diferentes, o que pode gerar problemas de processamento e eficiência no modelo.

Parar solucionar isso, são utilizadas técnicas como *padding* e *truncation*, que garantem que todas as sequências de entrada tenham um comprimento uniforme, permitindo que o modelo processe os dados de maneira consistente e eficiente. Vamos entender a diferença e usabilidade de cada um deles.

#### Padding

O padding é o processo de adicionar tokens (geralmente um token especial, como [PAD]) ao final ou ao início de uma sequência de texto para garantir que todas as sequências em um lote tenham o mesmo comprimento. Vamos entender isso a partir de um exemplo. Imagine que temos as seguintes sequências de texto:

- “Eu gosto de aprender”
- “Aprender é divertido”
- “Estudar”

Para processar essas sequências em um lote, precisamos que **todas tenham o mesmo comprimento**. Vamos supor que decidimos que todas devem ter 4 tokens. Aqui está como seria com padding, onde cada palavra é entendida como um token:

- “Eu gosto de aprender”
- “Aprender é divertido. [PAD]”
- “Estudar. [PAD] [PAD] [PAD]”

Dessa forma, todas as sequências têm o mesmo comprimento, facilitando o processamento em batch pelos modelos.

#### Truncation

O truncation é o processo de cortar uma sequência de texto para garantir que **ela não exceda um determinado comprimento**. Isso é útil quando temos textos muito longos e queremos garantir que todos os textos no lote tenham um comprimento máximo específico, economizando recursos computacionais e memória. Vamos utilizar o mesmo exemplo para entender como o truncation funciona.

Imagine que temos as seguintes sequências de texto:

- “Eu gosto de aprender”
- “Aprender é divertido”
- “Estudar”

Se decidirmos que o comprimento máximo é de 3 tokens, o truncamento funcionaria assim:

- “Eu gosto de” (truncado de “Eu gosto de aprender.”)
- “Aprender é” (truncado de “Aprender é divertido.”)
- “Estudar” (permanece o mesmo porque só tem 1 token)

Caso queira entender sobre a utilização do padding e truncation da biblioteca Transformers, consulte a documentação:

- [Padding and Truncation](https://huggingface.co/docs/transformers/pad_truncation)

## 03. Ajustando dados para o modelo

### Ajustando a variável alvo

In [ ]:
dataset_tokenizado['treino'].features['score']

In [ ]:
from datasets import ClassLabel

In [ ]:
dataset_tokenizado['treino'].unique('score')

In [ ]:
scores = ClassLabel(names=[str(i) for i in range(11)])

In [ ]:
scores

In [ ]:
def mapear_labels(dados):
    dados['label'] = scores.str2int(str(dados['score']))
    return dados

In [ ]:
dataset_tokenizado = dataset_tokenizado.map(mapear_labels, remove_columns='score')

In [ ]:
dataset_tokenizado = dataset_tokenizado.cast_column('label', scores)

In [ ]:
dataset_tokenizado

In [ ]:
dataset_tokenizado['treino'].features['label']

### Carregando o modelo pré-treinado

In [ ]:
from transformers import TFAutoModelForSequenceClassification

In [ ]:
id2label = {i: str(i) for i in range(11)}
label2id = {v: k for k, v in id2label.items()}

modelo = TFAutoModelForSequenceClassification.from_pretrained(
    checkpoint_modelo,
    num_labels = dataset_tokenizado['treino'].features['label'].num_classes,
    id2label=id2label,
    label2id=label2id,
    from_pt=True
)

In [ ]:
dados_treino = modelo.prepare_tf_dataset(
    dataset_tokenizado['treino'],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizador
)

dados_validacao = modelo.prepare_tf_dataset(
    dataset_tokenizado['teste'],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizador
)

### Para saber mais: diferentes métodos de carregamento de modelos

A biblioteca [`Transformers`](https://huggingface.co/docs/transformers/index) é uma ferramenta poderosa para o desenvolvimento de modelos de linguagem natural baseados em deep learning. Entre suas várias funcionalidades, as Auto Classes (Classes Automáticas) se destacam por simplificar e agilizar o uso de modelos pré-treinados, sendo capazes de identificar e carregar automaticamente o modelo correto com base em uma string identificadora do modelo. Vamos agora conhecer algumas.

#### Principais Auto Classes

- **AutoModel**: é utilizado para carregar modelos de aprendizado profundo para diversas tarefas de NLP (Processamento de Linguagem Natural). Ele automaticamente seleciona o tipo de modelo correto (como BERT, GPT-2, RoBERTa, etc.) baseado n onome do modelo fornecido.
- **AutoTokenizer**: carrega o tokenizador correspondente ao modelo. O tokenizador é responsável por converter o texto bruto em tokens que o modelo pode processar.
- **AutoConfig**: carrega a configuração do modelo, que contém parâmetros importantes como o número de camadas, tamanho do vocabulário, entre outros. É útil para inspeção ou para quando se deseja modificar algumas configurações antes de carregar o modelo.

#### Auto Classes específicas

Além das Auto Classes gerais, biblioteca também oferece várias Auto Classes específicas, cada uma projetada para diferentes tarefas de NLP. Considere a seguir as principais classes específicas e suas utilizações:

- **AutoModelForSequenceClassification**: utilizada para tarefas de classificação de sequência, como análise de sentimentos, detecção de spam e classificação de tópicos.
- **AutoModelForTokenClassification**: utilizada para tarefas de classificação de tokens, como reconhecimento de entidade nomeadas (NER) e marcação de partes do discurso (POS tagging).
- **AutoModelForQuestionAnswering**: utilizada para sistemas de perguntas e respostas, onde o modelo responde a perguntas baseadas em um contexto fornecido.
- **AutoModelForSeq2SeqLM**: utilizada para tarefas de tradução de linguagem, resumo de textos, e outras tarefas de sequência-para-sequência (seq2seq).
- **AutoModelForCausalLM**: utilizada para tarefas de geração de texto, como modelos de linguagem autoregressivos (e.g., GPT-2, GPT-3).
- **AutoModelForMaskedLM**: utilizada para tarefas de modelagem de linguagem com máscaras, como preenchimento de palavras mascaradas em um texto (e.g., BERT).
- **AutoModelForMultipleChoice**: utilizada para tarefas de escolha múltipla, onde o modelo escolhe a resposta correta entre várias opções fornecidas.
- **AutoModelForNextSentencePrediction**: utilizada para tarefas de predição da próxima sentença, como determinar se uma sentença segue logicamente outra.

#### Vantagens das Auto Classes

| **Benefício**           | **Descrição**                                                                                       |
|-------------------------|-----------------------------------------------------------------------------------------------------|
| **Simplicidade e Conveniência** | Elimina a necessidade de saber detalhes específicos sobre a arquitetura do modelo que se está carregando. Basta fornecer o nome do modelo. |
| **Flexibilidade**       | Funciona com uma ampla gama de modelos diferentes, permitindo fácil troca entre diferentes arquiteturas e experimentação. |
| **Consistência**        | Garante uma interface consistente, independente do modelo subjacente, o que facilita o desenvolvimento e a manutenção de código. |

Caso queira conhecer mais sobre os AutoModels e todas as suas possibilidades, veja a documentação oficial da biblioteca Transformers:

- [Auto Models - documentação Hugging Face](https://huggingface.co/docs/transformers/model_doc/auto)

## 04. Transferindo o aprendizado para um modelo

### Treinando um modelo

In [ ]:
from transformers import create_optimizer

In [ ]:
batch_size = 16
epocas = 2
batches_por_epoca = len(dataset_tokenizado['treino']) // batch_size
total_passos_treino = int(batches_por_epoca * epocas)
taxa_aprendizado = 2e-5

In [ ]:
otimizador, scheduler = create_optimizer(
    init_lr=taxa_aprendizado,
    num_warmup_steps=0,
    num_train_steps=total_passos_treino
)

modelo.compile(optimizer=otimizador)

In [ ]:
modelo.fit(dados_treino, validation_data=dados_validacao, epochs=epocas)

In [ ]:
resultados_avaliacao = modelo.evaluate(dados_validacao)
print(f"Loss: {resultados_avaliacao}")  # Loss: 1.6454880237579346

### Publicando o modelo no Hugging Face Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
modelo.push_to_hub('distilbert-pt-cased-essays-score')

In [ ]:
tokenizador.push_to_hub('distilbert-pt-cased-essays-score')

## 05. Colocando o modelo em produção

### Utilizando o modelo para previsão

In [ ]:
# modelo = TFAutoModelForSequenceClassification.from_pretrained('c3p0gan/distilbert-pt-cased-essays-score')
# tokenizador = AutoTokenizer.from_pretrained('c3p0gan/distilbert-pt-cased-essays-score')

In [ ]:
dados_redacoes['teste'].to_pandas()

In [ ]:
textos = [dados_redacoes['teste']['essay'][2], dados_redacoes['teste']['essay'][909]]

In [ ]:
import numpy as np

In [ ]:
textos_tokenizados = tokenizador(textos, return_tensors='np', padding='longest')

In [ ]:
resultados = modelo(textos_tokenizados).logits
resultados

In [ ]:
classificacao = np.argmax(resultados, axis=1)
print(classificacao)

In [ ]:
classificador = pipeline('text-classification', model='c3p0gan/distilbert-pt-cased-essays-score', framework='tf')

In [ ]:
classificador(textos)

### Para saber mais: resultado da previsão de um modelo de IA

Quando um modelo de IA realiza uma previsão, geralmente não fornece diretamente uma classe ou uma probabilidade. Em vez disso, são gerados um conjunto de valores numéricos chamados **logits**. Os logits podem ser qualquer número real, positivo ou negativo, e indicam a força e a direção da previsão do modelo para cada classe possível.

Para converter os logits em probabilidades compreensíveis, utiliza-se uma **função de ativação**. A mais comum é a softmax, que transforma os logits em valores entre 0 e 1, somando exatamente 1, tornando-os probabilidades (scores). Essas probabilidades indicam a confiança do modelo em cada classe.

Logits são importantes porque:

- Eles contêm informação sobre a confiança do modelo: logits mais altos indicam maior confiança.
- Facilita o cálculo da métrica perda (Loss): funções de perda, como a entropia cruzada, usam logits para calcular o erro do modelo durante o treinamento.
- Servem como base para a decisão final: antes de aplicar a função de ativação, os logits ajudam a entender como o modelo chega às suas conclusões.

## Para ir mais a fundo

[Transformers, documentação oficial](https://huggingface.co/docs/transformers/index)

[Datasets, documentação oficial](https://huggingface.co/docs/datasets/index)

[Gradio, documentação oficial](https://www.gradio.app/docs)

[curso de NLP com Hugging Face](https://huggingface.co/learn/nlp-course/chapter1/1)

[Notebooks de exemplos de projetos com Hugging Face, GitHub](https://github.com/huggingface/notebooks/tree/main/examples)